In [1]:
import pandas as pd
import urllib3
import requests
from bs4 import BeautifulSoup
from io import BytesIO
import gzip

In [2]:
df = pd.read_csv("input/raw/cc_mlb.csv")
df.head()

,Unnamed: 0,charset,digest,filename,languages,length,mime,mime-detected,offset,status,timestamp,url,urlkey
0,0,UTF-8,LQMEKH2VUTYILOQRFAIPGNA25VS4AX64,crawl-data/CC-MAIN-2019-13/segments/1552912203...,eng,34907,text/html,text/html,279018202,200,20190325053439,http://www.espn.com/mlb/story/_/id/10015217/ml...,"com,espn)/mlb/story/_/id/10015217/mlb-players-..."
1,1,UTF-8,BZMV7IMCPPK7WMRASVZADI3JPPFW3QYG,crawl-data/CC-MAIN-2019-13/segments/1552912201...,eng,33236,text/html,text/html,280546961,200,20190319033258,http://www.espn.com/mlb/story/_/id/10019043/co...,"com,espn)/mlb/story/_/id/10019043/colorado-roc..."
2,2,UTF-8,4MR466DR32C5T6B3U7D2RDZMAKZYKTEZ,crawl-data/CC-MAIN-2019-13/segments/1552912202...,eng,38336,text/html,text/html,295127884,200,20190323005221,http://www.espn.com/mlb/story/_/id/10042646/ja...,"com,espn)/mlb/story/_/id/10042646/javier-lopez..."
3,3,UTF-8,7Z5TT4LMUUFGOJYXRU4FM3NKAMLNMBCJ,crawl-data/CC-MAIN-2019-13/segments/1552912203...,eng,38751,text/html,text/html,283761478,200,20190323211500,http://www.espn.com/mlb/story/_/id/10074360/ta...,"com,espn)/mlb/story/_/id/10074360/tampa-bay-ra..."
4,4,UTF-8,RHSK6WX26QBULNJTEML6V4362US6664T,crawl-data/CC-MAIN-2019-13/segments/1552912202...,eng,30356,text/html,text/html,281365425,200,20190320054445,http://www.espn.com/mlb/story/_/id/10083830/se...,"com,espn)/mlb/story/_/id/10083830/seattle-mari..."


In [3]:
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    try:
        # We can then use the Range header to ask for just this set of bytes
        resp = requests.get(
            prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

        # The page is stored compressed (gzip) to save space
        # We can extract it using the GZIP library
        raw_data = BytesIO(resp.content)
        f = gzip.GzipFile(fileobj=raw_data)

    except OSError as e:
        print(e)
        return ""
        
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""

    if len(data):

        try:
            # data = data.decode("utf-8")
            # print(data)

            [warc, header, response] = data.strip().split(b'\r\n\r\n', 2)
            # print(warc, header)
        except Exception as e:
            print(e)
            return response

    return response

In [4]:
def get_text_from_html(html_content):
    text = ""
    soup = BeautifulSoup(html_content, features="html.parser")

    matches = soup.select(
        "section > article:first-child > div.container > div.article-body > p")

    for item in matches:
        text += item.get_text()+"\n"
    return text

In [5]:
text = ""
for idx, row in df.iterrows():
    if row['status'] == 200:
        html_content = download_page(row)

        print("[*] Retrieved %d bytes for %s" %
              (len(html_content), row['url']))
        if len(html_content) != 0:
            text += get_text_from_html(html_content)

[*] Retrieved 163003 bytes for http://www.espn.com/mlb/story/_/id/10015217/mlb-players-union-head-michael-weiner-dies-51
[*] Retrieved 164616 bytes for http://www.espn.com/mlb/story/_/id/10019043/colorado-rockies-latroy-hawkins-finalize-deal
[*] Retrieved 208845 bytes for http://www.espn.com/mlb/story/_/id/10042646/javier-lopez-re-signs-san-francisco-giants-three-years-13-million
[*] Retrieved 201985 bytes for http://www.espn.com/mlb/story/_/id/10074360/tampa-bay-rays-acquire-ryan-hanigan-heath-bell-three-team-trade
[*] Retrieved 146785 bytes for http://www.espn.com/mlb/story/_/id/10083830/seattle-mariners-sign-willie-bloomquist-2-year-deal
[*] Retrieved 238636 bytes for http://www.espn.com/mlb/story/_/id/10084018/masahiro-tanaka-not-made-available-mlb-teams-rakuten-golden-eagles
[*] Retrieved 208531 bytes for http://www.espn.com/mlb/story/_/id/10107900/curt-schilling-replacing-orel-hershiser-espn-sunday-night-baseball-booth
[*] Retrieved 173374 bytes for http://www.espn.com/mlb/story/

[*] Retrieved 164844 bytes for http://www.espn.com/mlb/story/_/id/11038984/minnesota-twins-selection-nick-gordon-emotional
[*] Retrieved 160694 bytes for http://www.espn.com/mlb/story/_/id/11044704/johan-santana-baltimore-orioles-tears-achilles-season
[*] Retrieved 172700 bytes for http://www.espn.com/mlb/story/_/id/11049084/philadelphia-phillies-place-mike-adams-dl-call-kenny-giles
[*] Retrieved 226427 bytes for http://www.espn.com/mlb/story/_/id/11051832/mitch-moreland-texas-rangers-ankle-surgery-miss-three-months
[*] Retrieved 159616 bytes for http://www.espn.com/mlb/story/_/id/11056097/tampa-bay-rays-replace-closer-grant-balfour-committee
[*] Retrieved 233116 bytes for http://www.espn.com/mlb/story/_/id/11091626/hall-famer-tony-gwynn-san-diego-padres-died?utm_source=hootsuite&utm_campaign=hootsuite
[*] Retrieved 190329 bytes for http://www.espn.com/mlb/story/_/id/11120049/san-diego-padres-fire-general-manager-josh-byrnes
[*] Retrieved 228240 bytes for http://www.espn.com/mlb/story/

[*] Retrieved 172647 bytes for http://www.espn.com/mlb/story/_/id/12279487/carlos-villanueva-agrees-minor-league-deal-st-louis-cardinals
[*] Retrieved 158523 bytes for http://www.espn.com/mlb/story/_/id/12320133/aroldis-chapman-cincinnati-reds-reaches-one-year-deal-avoids-arbitration
[*] Retrieved 157609 bytes for http://www.espn.com/mlb/story/_/id/12337085/cleveland-indians-lhp-bruce-chen-reach-minor-league-deal
[*] Retrieved 158156 bytes for http://www.espn.com/mlb/story/_/id/12340701/kenley-jansen-los-angeles-dodgers-8-12-weeks-foot-surgery
[*] Retrieved 171959 bytes for http://www.espn.com/mlb/story/_/id/12351883/major-league-baseball-announce-pace-play-rules
[*] Retrieved 172961 bytes for http://www.espn.com/mlb/story/_/id/12366824/bj-upton-atlanta-braves-go-melvin-upton-jr-2015
[*] Retrieved 186510 bytes for http://www.espn.com/mlb/story/_/id/12373346/joba-chamberlain-signs-1-year-deal-detroit-tigers
[*] Retrieved 154685 bytes for http://www.espn.com/mlb/story/_/id/12394519/juan-

[*] Retrieved 201009 bytes for http://www.espn.com/mlb/story/_/id/13437738/heart-recipient-throwing-first-pitch-fenway
[*] Retrieved 208342 bytes for http://www.espn.com/mlb/story/_/id/13439652/jose-valverde-suspended-80-games-positive-drug-test-minor-leagues
[*] Retrieved 190309 bytes for http://www.espn.com/mlb/story/_/id/13449370/peter-moylan-makes-back-majors-atlanta-braves
[*] Retrieved 165380 bytes for http://www.espn.com/mlb/story/_/id/13454628/toronto-blue-jays-everything-baseball-aint
[*] Retrieved 181613 bytes for http://www.espn.com/mlb/story/_/id/13463881/juan-marichal-hit-john-roseboro-bat-ugly-baseball-brawl-50-years-ago
[*] Retrieved 153782 bytes for http://www.espn.com/mlb/story/_/id/13470907/san-diego-padres-promote-outfielder-travis-jankowski-triple-a
[*] Retrieved 187895 bytes for http://www.espn.com/mlb/story/_/id/13477400/pittsburgh-pirates-sign-travis-snider-minor-league-deal
[*] Retrieved 159749 bytes for http://www.espn.com/mlb/story/_/id/13485353/mlb-players-un

[*] Retrieved 207838 bytes for http://www.espn.com/mlb/story/_/id/14424328/lefty-ross-detwiler-agrees-minor-league-deal-indians
[*] Retrieved 235037 bytes for http://www.espn.com/mlb/story/_/id/14428286/logan-kensing-nate-schierholtz-agree-detroit-tigers
[*] Retrieved 197035 bytes for http://www.espn.com/mlb/story/_/id/14454106/reds-deal-closer-aroldis-chapman-yankees
[*] Retrieved 156613 bytes for http://www.espn.com/mlb/story/_/id/14455628/oakland-athletics-henderson-alvarez-agree-1-year-deal
[*] Retrieved 191549 bytes for http://www.espn.com/mlb/story/_/id/14515169/los-angeles-dodgers-re-sign-brandon-beachy
[*] Retrieved 197638 bytes for http://www.espn.com/mlb/story/_/id/14515952/washington-nationals-announce-deal-daniel-murphy-pays-just-8m-2016
[*] Retrieved 241622 bytes for http://www.espn.com/mlb/story/_/id/14568028/odd-troubling-state-national-league
[*] Retrieved 162896 bytes for http://www.espn.com/mlb/story/_/id/14579447/jake-arrieta-chicago-cubs-55-million-apart-salary-nego

...

[*] Retrieved 187546 bytes for http://www.espn.com/mlb/story/_/id/6808219/kansas-city-royals-sign-dominican-prospect-adalberto-mondesi
[*] Retrieved 224211 bytes for http://www.espn.com/mlb/story/_/id/6808357/pittsburgh-pirates-file-complaint-umpire-jerry-meals-call-19-inning-loss
[*] Retrieved 157097 bytes for http://www.espn.com/mlb/story/_/id/6831225/cincinnati-reds-scott-rolen-surgery-repair-left-shoulder
[*] Retrieved 157011 bytes for http://www.espn.com/mlb/story/_/id/6839125/mlb-warns-players-deer-antler-spray-report-says
[*] Retrieved 157909 bytes for http://www.espn.com/mlb/story/_/id/6841751/san-francisco-giants-bruce-rips-radio-host-racist-ramon-ramirez-remark
[*] Retrieved 155379 bytes for http://www.espn.com/mlb/story/_/id/6868616/pittsburgh-pirates-activate-left-fielder-jose-tabata-option-third-baseman-pedro-alvarez
[*] Retrieved 198162 bytes for http://www.espn.com/mlb/story/_/id/6885807/kevin-correia-pittsburgh-pirates-put-15-day-disabled-list
[*] Retrieved 207521 bytes

[*] Retrieved 161149 bytes for http://www.espn.com/mlb/story/_/id/7498279/baltimore-orioles-sign-infielder-wilson-betemit-two-year-deal
[*] Retrieved 200787 bytes for http://www.espn.com/mlb/story/_/id/7498284/detroit-tigers-reportedly-land-prince-fielder-214m-offer
[*] Retrieved 176280 bytes for http://www.espn.com/mlb/story/_/id/7499291/cincicincinnati-reds-agree-minor-league-deal-veteran-willie-harris
[*] Retrieved 211816 bytes for http://www.espn.com/mlb/story/_/id/7506666/cleveland-indians-put-fausto-carmona-roberto-hernandez-heredia-restricted-list
[*] Retrieved 225832 bytes for http://www.espn.com/mlb/story/_/id/7522833/cleveland-indians-get-international-league-mvp-russ-canzler-tampa-bay-rays
[*] Retrieved 205743 bytes for http://www.espn.com/mlb/story/_/id/7544803/caribbean-series-francisco-liriano-andy-dirks-lift-dominican-republic-escogido-leones
[*] Retrieved 192266 bytes for http://www.espn.com/mlb/story/_/id/7551464/danny-clyburn-jr-former-major-league-outfielder-shot-kil

[*] Retrieved 184584 bytes for http://www.espn.com/mlb/story/_/id/8458298/detroit-tigers-miguel-cabrera-wins-first-triple-crown-1967
[*] Retrieved 156475 bytes for http://www.espn.com/mlb/story/_/id/8461744/seattle-mariners-fire-hitting-coach-chris-chambliss
[*] Retrieved 224615 bytes for http://www.espn.com/mlb/story/_/id/8462688/kansas-city-royals-part-ways-hitting-coach-kevin-seitzer
[*] Retrieved 154391 bytes for http://www.espn.com/mlb/story/_/id/8463398/washington-nationals-gio-gonzalez-wins-warren-spahn-award
[*] Retrieved 176068 bytes for http://www.espn.com/mlb/story/_/id/8465102/jeter-era-ending
[*] Retrieved 160245 bytes for http://www.espn.com/mlb/story/_/id/8466928/minnesota-twins-gm-terry-ryan-removes-interim-tag-title
[*] Retrieved 196114 bytes for http://www.espn.com/mlb/story/_/id/8480395/2012-alds-detroit-tigers-al-alburquerque-says-kiss-meant-no-disrespect
[*] Retrieved 194993 bytes for http://www.espn.com/mlb/story/_/id/8518040/arizona-diamondbacks-fire-1b-coach-eri

[*] Retrieved 160569 bytes for http://www.espn.com/mlb/story/_/id/9387977/johnny-vander-meer-consecutive-no-hitters-75-years-ago
[*] Retrieved 188460 bytes for http://www.espn.com/mlb/story/_/id/9404248/hamstring-forces-san-diego-padres-everth-cabrera-disabled-list
[*] Retrieved 165766 bytes for http://www.espn.com/mlb/story/_/id/9442231/bryce-harper-washington-nationals-returns-missing-31-games
[*] Retrieved 246064 bytes for http://www.espn.com/mlb/story/_/id/9452014/pitcher-tomohiro-anraku-future-japanese-baseball-espn-magazine
[*] Retrieved 220514 bytes for http://www.espn.com/mlb/story/_/id/9475310/houston-astros-sign-jose-altuve-2017
[*] Retrieved 164167 bytes for http://www.espn.com/mlb/story/_/id/9482248/white-sox-pitcher-chris-sale-skinny-stature-lasting-career-espn-magazine
[*] Retrieved 177077 bytes for http://www.espn.com/mlb/story/_/id/9500252/ryan-braun-milwaukee-brewers-suspended-remainder-2013-season
[*] Retrieved 171353 bytes for http://www.espn.com/mlb/story/_/id/95037

[*] Retrieved 200722 bytes for http://www.espn.com/mlb/story/_/page/rumblings120608/five-teams-shocked-most-2012
[*] Retrieved 193663 bytes for http://www.espn.com/mlb/story/_/page/seasonpreview_mlbteampredictions/mlb-team-predictions-2016-season
[*] Retrieved 223783 bytes for http://www.espn.com/mlb/story/_/page/SeasonpreviewMLB_2017predictions/2017-mlb-experts-mlb-team-predictions
[*] Retrieved 238422 bytes for http://www.espn.com/mlb/story/_/page/springtraining_daviddahl/david-dahl-lost-spleen-outfield-collision-rising-colorado-rockies
[*] Retrieved 155648 bytes for http://www.espn.com/mlb/story/_/page/springtraining_lucasgiolito/washington-nationals-lucas-giolito-change-national-league-east
[*] Retrieved 179070 bytes for http://www.espn.com/mlb/story/_/page/springtraining_yearofthecubs/st-louis-cardinals-chime-being-year-chicago-cubs


In [6]:
#2 change file name here based on sub-topic
file = open("./input/mr/cc_mlb.txt", "w", encoding="utf-8") 
file.write(text)
file.close()